<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
import numpy as np
import pandas as pd
import gensim
import os, sys, re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim

import spacy
nlp = spacy.load('en_core_web_lg')


/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/thinc/neural/train.py:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from .optimizers import Adam, linear_decay
/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/thinc/check.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, Sized, Iterable, Callable
/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/thinc/check.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecate

In [2]:
path = './data/amazon/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv'

In [3]:
df = pd.read_csv(path)

In [4]:
print(df.shape)
df.head(3)

(28332, 24)


,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."


In [5]:
columns = ['id', 'brand', 'primaryCategories', 'reviews.text', 'reviews.title']

df = df[columns]

In [6]:
df.head(3)

,id,brand,primaryCategories,reviews.text,reviews.title
0,AVpgNzjwLJeJML43Kpxn,Amazonbasics,Health & Beauty,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...
1,AVpgNzjwLJeJML43Kpxn,Amazonbasics,Health & Beauty,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...
2,AVpgNzjwLJeJML43Kpxn,Amazonbasics,Health & Beauty,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...


In [7]:
def tokenize(text):
    lemmas = []
    
    doc = nlp(text)
    

    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_!= 'PRON'):
            lemmas.append(token.lemma_)
    
    return lemmas

In [8]:
df['tokens'] = df['reviews.text'].apply(tokenize)

In [9]:
df['tokens'][0]

['order',
 '3',
 'item',
 'bad',
 'quality',
 'miss',
 'backup',
 'spring',
 'pcs',
 'aluminum',
 'battery',
 'work']

In [10]:
id2word = corpora.Dictionary((df['tokens']))

In [11]:
id2word.filter_extremes(no_below=10, no_above=0.75)

In [12]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [13]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=4
                  )

In [14]:
lda.print_topics()

[(0,
  '0.186*"love" + 0.059*"use" + 0.058*"gift" + 0.052*"buy" + 0.050*"easy" + 0.027*"Christmas" + 0.025*"son" + 0.020*"watch" + 0.019*"purchase" + 0.017*"absolutely"'),
 (1,
  '0.022*"come" + 0.020*"slow" + 0.018*"open" + 0.016*"box" + 0.015*"take" + 0.013*"home" + 0.012*"thing" + 0.012*"little" + 0.011*"like" + 0.011*"try"'),
 (2,
  '0.091*"kid" + 0.062*"tablet" + 0.032*"time" + 0.029*"good" + 0.022*"control" + 0.021*"free" + 0.018*"parental" + 0.017*"great" + 0.017*"app" + 0.016*"set"'),
 (3,
  '0.076*"app" + 0.060*"great" + 0.059*"love" + 0.048*"kid" + 0.039*"game" + 0.030*"easy" + 0.027*"learn" + 0.026*"use" + 0.025*"child" + 0.023*"tablet"'),
 (4,
  '0.137*"tablet" + 0.036*"old" + 0.028*"year" + 0.027*"great" + 0.026*"love" + 0.026*"game" + 0.018*"use" + 0.016*"play" + 0.015*"watch" + 0.015*"buy"'),
 (5,
  '0.068*"year" + 0.065*"old" + 0.044*"buy" + 0.039*"love" + 0.028*"charge" + 0.027*"grandson" + 0.023*"2" + 0.019*"3" + 0.018*"5" + 0.017*"play"'),
 (6,
  '0.097*"kindle" + 0.

In [15]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [16]:
topics = [' '.join(t[0:5]) for t in words]

In [17]:
for t in topics: 
    print(t)
    print("\n")

love use gift buy easy


come slow open box take


kid tablet time good control


app great love kid game


tablet old year great love


year old buy love charge


kindle user friendly great fire


product great recommend good Amazon


buy purchase enjoy daughter love


easy read screen light size


tablet Amazon great app use


battery long brand buy Amazon


work fine purchase expect get


great price good work battery


book read Kindle Fire use




In [18]:
pyLDAvis.enable_notebook()

In [19]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11    -0.206115  0.102539       1        1  19.202372
10    -0.039340 -0.073971       2        1   8.519008
13    -0.316848 -0.151555       3        1   7.208590
4      0.114016 -0.094035       4        1   7.200968
14     0.110277  0.110996       5        1   6.891551
9      0.049926  0.089152       6        1   6.682895
5      0.129850  0.026610       7        1   6.456927
2      0.018767 -0.131025       8        1   5.954944
7     -0.204832 -0.041159       9        1   5.564732
1     -0.066254  0.180342      10        1   5.004206
0      0.175087 -0.028393      11        1   4.522045
3      0.084328 -0.227938      12        1   4.433571
8      0.151428  0.047294      13        1   4.289698
12    -0.080340  0.196482      14        1   4.185395
6      0.080050 -0.005339      15        1   3.883090, topic_info=     Category         Freq          Term        Total  loglift  logprob
3     Default  9256.000000       battery  9256.000000  30.0000  30.0000
18    Default  9127.000000         great  9127.000000  29.0000  29.0000
15    Default  5113.000000         price  5113.000000  28.0000  28.0000
1589  Default  7098.000000        tablet  7098.000000  27.0000  27.0000
79    Default  6193.000000          love  6193.000000  26.0000  26.0000
11    Default  2527.000000       product  2527.000000  25.0000  25.0000
29    Default  6653.000000          good  6653.000000  24.0000  24.0000
8     Default  4407.000000          work  4407.000000  23.0000  23.0000
161   Default  2932.000000           kid  2932.000000  22.0000  22.0000
1429  Default  1427.000000        kindle  1427.000000  21.0000  21.0000
439   Default  2535.000000           old  2535.000000  20.0000  20.0000
1118  Default  2395.000000           app  2395.000000  19.0000  19.0000
91    Default  2300.000000          year  2300.000000  18.0000  18.0000
959   Default  1867.000000          book  1867.000000  17.0000  17.0000
28    Default  3361.000000          easy  3361.000000  16.0000  16.0000
16    Default  2094.000000         brand  2094.000000  15.0000  15.0000
117   Default  1195.000000          gift  1195.000000  14.0000  14.0000
26    Default  5641.000000           buy  5641.000000  13.0000  13.0000
637   Default  2086.000000          read  2086.000000  12.0000  12.0000
153   Default  1956.000000          game  1956.000000  11.0000  11.0000
83    Default  4854.000000           use  4854.000000  10.0000  10.0000
1718  Default  1729.000000        Kindle  1729.000000   9.0000   9.0000
34    Default  2550.000000      purchase  2550.000000   8.0000   8.0000
338   Default   962.000000         value   962.000000   7.0000   7.0000
1192  Default  1691.000000        screen  1691.000000   6.0000   6.0000
800   Default  1039.000000      daughter  1039.000000   5.0000   5.0000
20    Default  2596.000000          long  2596.000000   4.0000   4.0000
1455  Default   815.000000         enjoy   815.000000   3.0000   3.0000
78    Default   997.000000         light   997.000000   2.0000   2.0000
143   Default  1334.000000     recommend  1334.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
1477  Topic15    73.858994        choose   154.434738   2.5109  -5.0937
1074  Topic15   170.256348        access   461.686981   2.2510  -4.2586
1260  Topic15   106.242294       content   267.266174   2.3260  -4.7302
642   Topic15   251.434616          fire  1096.184814   1.7761  -3.8687
298   Topic15   102.162033        option   329.837524   2.0765  -4.7693
84    Topic15   200.070206          want  1264.823730   1.4045  -4.0972
95    Topic15   122.783752          able   583.376831   1.6901  -4.5855
959   Topic15   243.836090          book  1867.759644   1.2125  -3.8994
175   Topic15    39.259918  subscription    98.908310   2.3245  -5.7257
665   Topic15    67.669434         allow   286.375854   1.8059  -5.1813
349   Topic15  

In [20]:
def compute_coherence_values(dictionary, corpus, path, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    path : path to input texts
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    tokens = list(doc_stream(path))
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            stream = doc_stream(path)
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

In [21]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus,
                                                        path= path,
                                                        start=2, 
                                                        limit=40, 
                                                        step=6)

NameError: name 'doc_stream' is not defined

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling